# D3D IO Module

The following example will familiarize the user with using the d3d toolbox in MHKiT-Python. `d3d` can be used to plot the data from a NetCDF output from Delft3D [SNL-D3D-CEC-FM](https://github.com/MHKiT-Software/MHKiT-Python). The example will walk through a flume case with a turbine. The flume is 18 m long and 4 m wide with the turbine placed at 6 m along the length and 3 m along the width. The turbine used for this simulation has a circular cross-section with a diameter of 0.7m, a thrust coefficient of 0.72, and a power coefficient of 0.45. The simulation was run with 5 depth layers and 3-time intervals.

This example will show how to create a centerline plot at a desired depth for different variables outputs of Delft3D using matplotlib. It will also show how to make a contour plot of a given plane or depth for the variable output by Delft3D and how to use those variables to calculate turbulence intensity. This module can be helpful to visualize the wake of a turbine and help predict how a turbine will affect the surrounding area. 

Start by importing the necessary python packages and MHKiT module.  

In [ ]:
from os.path import abspath, dirname, join, normpath, relpath
from mhkit.river.io import d3d 
import scipy.interpolate as interp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import netCDF4

## Loading Data from Delft3D as a NetCDF 

 A NetCDF file has been saved in the [\\MHKiT-Python\\examples\\data\\river\\d3d](https://github.com/browniea/MHKiT-Python/tree/d3d/examples/data/river/d3d) directory of a simple flume case with a turbine for refrence data. Here we are inputting strings file path `datadir`, and file name `filename` so the NetCDF file can be saved as a NetCDF object using the python pack `netCDF4` under the variable  `d3d_data`.    
 
There are many variables saved in the NetCDF object `d3d_data`. The function `d3d_data.variables.keys()` returns a dictionary of the avalible data. Here we look at the dictionary keys output to see all the variables we can plot. 

In [ ]:
datadir = normpath(join(relpath(join('data', 'river', 'd3d'))))
filename= 'turbineTest_map.nc'
d3d_data = netCDF4.Dataset(join(datadir,filename)) 

for var in d3d_data.variables.keys():
    try: 
        d3d_data[var].long_name
    except:
        print(f'"{var}"')        
    else:
        print(f'"{var}": {d3d_data[var].long_name}')

## Dataframe from NetCDF
First the variables to be pulled from the NetCDF file are defined and saved under the name `variable`. The variable `'ucx'` is velocity in the x-direction. (see advalible data in previos block output). The `d3d` function `get_all_data_points` will pull all the raw data from the NetCDF file for the specified variable at a specific time index in a dataframe `var_data_df`. The `time_index` can be used to pull data from a different instance in time. The first index is 0 and the user can use -1 to select the last time_index in the data set. The defalt `time_index` is set to -1.The last `time_index` can also be found by finding array dimension of the input variable, as shown the max valid value for `time_index` is output. 

In [ ]:
variable= 'ucx' 

max_time_index= len(d3d_data.variables[variable][:])

print(f'max time_index:{max_time_index}')

# All data in  NetCDF File
var_data_df= d3d.get_all_data_points(d3d_data, variable, time_index=-1)

## Plotting a Line Plot Between Two Centerline Points
To veiw the maximum effects of the tubine we will creat a aray down the center of our flume. We will then interpolate the grided data results onto our centerline array. 

For this line plot we will find the centerline of the flume. The centerline points are found by taking the average of the maximum and minimum value for the width, `y`, and height, `z`, of the flume. We will use The function `create_points` to create our centerline array to interpolate over with the input of one array, `x`, and two points, `y` and `z`. `create_points` will then  output a dataframe, `cline_points`, with keys `'x'`, `'y'`,and `'z'`, of points. 

In [ ]:
xmin=var_data_df.x.max()
xmax=var_data_df.x.min()

ymin=var_data_df.y.max()
ymax=var_data_df.y.min()

zmin=var_data_df.z.max()
zmax=var_data_df.z.min()

x = np.linspace(xmin, xmax, num=100)
y = np.mean([ymin,ymax])
z = np.mean([zmin,zmax])

cline_points = d3d.create_points(x, y, z)

Next the variable `ucx` is interpolated over the `cline_points` using sipy interp function, and saved as `cline_variable`. The results are then plotted for  velocity in the x direction, `ucx`, along the length of the flume, `x`.

In [ ]:
cline_variable = interp.griddata(var_data_df[['x','y','z']], 
                     var_data_df[variable],
                     cline_points[['x','y','z']])

Type='Centerline'
plt.plot(x, cline_variable)
plt.grid()
plt.xlabel('x (m)')
plt.ylabel(f'{d3d_data.variables[variable].long_name} [{d3d_data.variables[variable].units}]')
plt.title(f'{Type} {d3d_data.variables[variable].long_name}')

## Contour Plot for a Given Sigma Layer 

Sometimes it will be useful to plot only the raw data. Using `get_layer_data` a single layer of raw data will be retrieved from the NetCDF object. The `d3d` function ,`get_layer_data` takes 3 inputs, the netCDFobject (`d3d_data`), the vaiable name (`variable`) as a string, and the the layer to retrieve (`layer`) as an integer. The variable is set to the velocity in the x direction, `'ucx'`, and the layer of data to plot as 2. Since there are 5 sigma layers in this example layer 2 is approximatly the middle layer.  `layer`  works as an idex that begins at begins at 0 and ends at 4.  The `get_layer_data` then outputs a dataframe `layer_data` with the keys `'x'`, `'y'`, and '`v`' as the length, width, and value of the specified variable in this case velocity in the x direction, 'ucx'.  

To plot the data the maximum and minimum, "max_plot_v" and "min_plot_v", values are defined to limit bounds of the color bar to show the value of the specified variable, in this case it's velocity in the x direction. The type of plot is also defined as 'contour' to add to the title of the plot.

In [ ]:
variable= 'ucx'
layer= 2
layer_data = d3d.get_layer_data(d3d_data, variable, layer)

max_plot_v= 1.5
min_plot_v=.5
  
Type= 'Contour'
plt.figure()
contour_plot = plt.tricontourf(layer_data.x ,layer_data.y, layer_data.v, vmin=min_plot_v,vmax=max_plot_v,
                               levels=np.linspace(min_plot_v,max_plot_v,10))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title(f'{Type} Layer: {layer} {d3d_data.variables[variable].long_name}')
cbar= plt.colorbar(contour_plot)
cbar.set_label(f'{d3d_data.variables[variable].long_name} [{d3d_data.variables[variable].units}]')

## Plotting a Contour Plot layer 

If you wanted to look at a contour plot of a specific depth you can create a contour plot over given points. Expanding on the centerline plot points we can create an array along the width of the flume, `y_contour` to add to the array along the length of the flume, `x`, and the point for the depth we want to look at `z`.  `create_points` then outputs a dataframe '`contour_points`' of points to calculate the contour values over with the keys `'x'`, `'y'`,and `'z'`.

In [ ]:
x = np.linspace(xmin, xmax, num=100)
y_contour = np.linspace(ymin, ymax, num=40)
z = np.mean([zmin,zmax])

contour_points = d3d.create_points(x, y_contour, z) 

Next the data is interpolated on to the points created saved under the variable name `'contour_variable'`. 

In [ ]:
contour_variable = interp.griddata(var_data_df[['x','y','z']], 
                                                     var_data_df[variable],
                                                     contour_points[['x','y','z']])

The results are then plotted. The minimum and maximum values show on the graph are defined by "max_plot_v" and "min_plot_v" as used in the previous example. The contour plot of the velocity is then output at the center depth of the flume. 

In [ ]:
max_plot_v= 1.5
min_plot_v=.5

#Plot contour
Type= 'Contour'
plt.figure()
contour_plot = plt.tricontourf(contour_points.x,contour_points.y,contour_variable,
                               vmin=min_plot_v,vmax=max_plot_v,levels=np.linspace(min_plot_v,max_plot_v,10))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title(f'{Type} {d3d_data.variables[variable].long_name}')
cbar= plt.colorbar(contour_plot)
cbar.set_label(f'{d3d_data.variables[variable].long_name} [{d3d_data.variables[variable].units}]')

## Contour Plot of Turbulent Intensity 
Turbulent Intensity is the ratio of the magnitude of turbulent velocity to total velocity. The function `turbulent_intensity`  pulls variables 'ucx', 'ucy', 'ucz', and 'turkin1' as the velocity in the x, y, z and turbulence respetivly. The function then calculates and outputs the turbulent intensity, `turbulent_intensity`, for any given time_index in the data_frame `TI`. The `TI` dataframe also includes the `'x'`,`'y'`, and '`z'` location, turbulent kinetic energy `'turkin1'`, and velocity in the `'ucx'`, `'ucy'`, and `'ucz'` direction. In this example it is calculating the turbulent intensity over the same contour_points used above, however it can also calculate over 'cells', the coordinate system for the raw velocity data, or 'faces', the coordinate system for the raw turbulence data. If nothing is specified for 'points', 'cells' is the defalt coordiante system.    

Following the same format as the previous two contour plots the limits of the maximum and minimum value are defined by user as well as a string for the type of plot. The code then outputs a contour plot of the turbulent intensity.

In [ ]:
TI= d3d.turbulent_intensity(d3d_data, contour_points)

max_plot_v= 0.2
min_plot_v=-0.2

Type= 'Contour'
plt.figure()
contour_plot = plt.tricontourf(contour_points.x, contour_points.y, TI.turbulent_intensity,
                               vmin=min_plot_v, vmax=max_plot_v,levels=np.linspace(min_plot_v,max_plot_v,10))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title(f'{Type} {d3d_data.variables[variable].long_name}')
cbar= plt.colorbar(contour_plot)
cbar.set_label(f'{d3d_data.variables[variable].long_name} [{d3d_data.variables[variable].units}]')

TI

## Comparing two Face Data to Cell Data 
In Delft3d ther is a stager where some vaiables are stored on the faces and some are stored on the cells and this function allows you to interpolate the date on the faces to the ce3ll locations or vis viersa.  
Similar to the `turbulent_intensity` function shown above the `grid data` function is used to interpolate data sets on to the same grid. This is useful for camparing variables such as velocity and turbulence where the raw data points are not on the same.  For this example, we input the variable names, 'ucx' and 'turkin1', the NetCDF object, 'd3d_data', and the corrdinate system  'faces'. The output is an array, ` Var`, with the interpolated data. The data can be called as shown with ‘Var.’ followed by the variable name.   

#claculate Turbulent intensity, 

In [ ]:
variables= ['ucx', 'turkin1']

Var= d3d.grid_data(d3d_data, variables, points= 'faces')

Var